# explanation #

# necessity #
confirm that you can run "gfortran" and "f2py". 

# make libraries #

execute
```
$ make
```
and confirm that 
```
  mylib.so
  mymod.so
  myext.so
```
are in the directory. 


# errors in jupyter #
If an error occurs in running ipynb, then shutdown the process and restart it. 
